# Add calculated ionization rates to KROME's datafile 

In [1]:
from Photon_Ionization import *
import yt.units as u

In [2]:
distance = np.arange(8)+1
dic = {'alpha':0.1, 'beta':100, 'mdot':3}
Dat = compare(distance, dic)

## Column density vs. Ionization rate

### Column density vs. Number density

KROME assumes the total number of $\text{H/He}$ atoms in different molecular clouds as a constant, and therefore
$$
N^i=1.87\times{10}^{21}\left(\frac{n^i}{1000}\right)^{2/3}\text{ cm}^{-2}
$$

In [3]:
def num2col0(nH, nHe):
    NH = 1.87e21*(nH/1000)**(2/3)
    NHe = 1.87e21*(nHe/1000)**(2/3)
    return(NH, NHe)
def col2num0(NH, NHe):
    nH = 1000*(NH/1.87e21)**(3/2)
    nHe = 1000*(NHe/1.87e21)**(3/2)
    return(nH, nHe)

In [4]:
'''
dataH = np.loadtxt('ratexH.dat')
dataHe = np.loadtxt('ratexHe.dat')
dataH[:,1] += dataH[:,0] 
dataH = 10**dataH
dataHe[:,1] += dataHe[:,0] 
dataHe = 10**dataHe

daH, daHe = dataH.copy(), dataHe.copy()
'''

"\ndataH = np.loadtxt('ratexH.dat')\ndataHe = np.loadtxt('ratexHe.dat')\ndataH[:,1] += dataH[:,0] \ndataH = 10**dataH\ndataHe[:,1] += dataHe[:,0] \ndataHe = 10**dataHe\n\ndaH, daHe = dataH.copy(), dataHe.copy()\n"

In [5]:
'''
ion = np.array([])
for NH, NHe in daH[:,0:2]:
    val, err = quad(lambda epsilon:F_E(epsilon)/epsilon*np.exp(-H.cross(epsilon)*NH-He.cross(epsilon)*NHe)*H.cross(epsilon), 2000, 10000)
    ion = np.append(ion, val)

for i in range(len(ion)):
    print(np.log10(ion[i]), np.log10(daH[:,-1][i]))
'''

'\nion = np.array([])\nfor NH, NHe in daH[:,0:2]:\n    val, err = quad(lambda epsilon:F_E(epsilon)/epsilon*np.exp(-H.cross(epsilon)*NH-He.cross(epsilon)*NHe)*H.cross(epsilon), 2000, 10000)\n    ion = np.append(ion, val)\n\nfor i in range(len(ion)):\n    print(np.log10(ion[i]), np.log10(daH[:,-1][i]))\n'

However, if we consider the Jeans length as Latif+2015 did, the relationship between the column density and the number density is totally different
$$N^i=n^i\sqrt{\frac{\pi kT}{G\rho\mu m_H}}$$
Thus
$$
\frac{N_H}{N_{He}}=\frac{n_H}{n_{He}}\equiv\chi
$$
Furthurmore
$$
\rho\mu m_H=\frac{(m_Hn_H+m_{He}n_{He})^2}{n_H+n_{He}}=n_H\frac{(m_H\chi+m_{He})^2}{\chi+1}
$$

In [5]:
def num2col(nH, nHe):
    chi = nH/nHe
    A = np.sqrt(np.pi*k*T/G)
    B = np.sqrt((mH*chi+mHe)**2/(chi+1)*nH)
    return(nH*A/B, nHe*A/B)
def col2num(NH, NHe):
    chi = NH/NHe
    C = np.sqrt(np.pi*k*T/G/(mH*chi+mHe)**2*(chi+1))
    nH = (NH/C)**2
    nHe = nH/chi
    return(nH, nHe)

- For the simple case, we directly utilize a radius of 1 pc for the molecular cloud

In [23]:
print('The column density (for hydrogen atoms) is {:e} cm^-2'.format((num2col(2e4,2e3)[0])))
print('The effective radius is {:f} pc'.format((num2col(2e4,2e3)[0]/2e4*u.cm).in_units('pc')))

The column density (for hydrogen atoms) is 2.045590e+22 cm^-2
The effective radius is 0.331465 pc


In [10]:
dataH = np.loadtxt('ratexH.dat')
dataHe = np.loadtxt('ratexHe.dat')
daH, daHe = dataH.copy(), dataHe.copy()

daH[:,1] += daH[:,0] 
daH = 10**daH
daHe[:,1] += daHe[:,0] 
daHe = 10**daHe

#daH[:,0], daH[:,1] = col2num(daH[:,0], daH[:,1])
#daHe[:,0], daHe[:,1] = col2num(daHe[:,0], daHe[:,1])

#cloud size - 1 pc
#convert = (u.pc/u.cm).in_cgs()

#cloud size - 5 pc
convert = (5*u.pc/u.cm).in_cgs()

#cloud size - 0.2 pc
#convert = (0.2*u.pc/u.cm).in_cgs()
daH[:,0], daH[:,1] = 1/convert*(daH[:,0], daH[:,1])
daHe[:,0], daHe[:,1] = 1/convert*(daHe[:,0], daHe[:,1])

In [3]:
dataH = np.loadtxt('ratexH.dat')
dataHe = np.loadtxt('ratexHe.dat')
daH, daHe = dataH.copy(), dataHe.copy()

daH[:,1] += daH[:,0] 
daH = 10**daH
daHe[:,1] += daHe[:,0] 
daHe = 10**daHe

In [29]:
'''
ion = np.array([])
for nH, nHe in daH[:,0:2]:
    ion = np.append(ion, Ion_p(H, He, H, None, nH, nHe))
for i in range(len(ion)):
    print(np.log10(ion[i]), np.log10(daH[:,-1][i]))
'''

'\nion = np.array([])\nfor nH, nHe in daH[:,0:2]:\n    ion = np.append(ion, Ion_p(H, He, H, None, nH, nHe))\nfor i in range(len(ion)):\n    print(np.log10(ion[i]), np.log10(daH[:,-1][i]))\n'

In [7]:
for da in Dat:
    ion = np.array([])
    da.intro()
    print(da.distance_kpc)
    for NH, NHe in daH[:,0:2]:
        #ion = np.append(ion, Ion_x(nH, nHe, H, He, H, He, da))
        ion = np.append(ion, Ion_x(NH, NHe, H, He, H, He, spec=da, N=True))
    '''
    for i in range(len(ion)):
        if i%30==5:
            print()
            print(np.log10(daH[i, 0]), np.log10(daH[i,1]), np.log10(ion[i]), np.log10(daH[:,-1][i]))
    '''
    
    f = open('/Users/chang/X-ray-chemistry/KROME/AGN_pc/rate_dis/ratexH' + str(int(da.distance_kpc))+'.dat', 'w')
    f.write('# KROME file, XRAY H rate\n')
    f.write('# Distance = ' + str(int(da.distance_kpc)) + ' kpc\n')
    f.write('# Xray rate including primary H ionization in 1/s\n')
    f.write('# first line indicates the size of the table (30x30)\n')
    f.write('# then first column is the parameter log10(nH), where nH is the H column density in cm-2\n')
    f.write('# second column is log10(xHe), where xHe = nHe/nH both in cm-2\n')
    f.write('# third column is log10(rate), where rate is in units of 1/s\n')
    f.write('30, 30')
    for i in range(len(ion)):
        if i%30==0:
            f.write('\n')
        f.writelines([str(dataH[i, 0]) + ' ' + str(dataH[i,1]) + ' ' + str(np.log10(ion[i])) + '\n'])
    f.close()

************************
alpha =  0.1
beta =  100
mdot =  3.0
************************
8.0
************************
alpha =  0.1
beta =  100
mdot =  3.0
************************
7.0
************************
alpha =  0.1
beta =  100
mdot =  3.0
************************
6.0
************************
alpha =  0.1
beta =  100
mdot =  3.0
************************
5.0
************************
alpha =  0.1
beta =  100
mdot =  3.0
************************
4.0
************************
alpha =  0.1
beta =  100
mdot =  3.0
************************
3.0
************************
alpha =  0.1
beta =  100
mdot =  3.0
************************
2.0
************************
alpha =  0.1
beta =  100
mdot =  3.0
************************
1.0


In [8]:
for da in Dat:
    ion = np.array([])
    da.intro()
    print(da.distance_kpc)
    for NH, NHe in daHe[:,0:2]:
        #ion = np.append(ion, Ion_x(nHe, nH, H, He, He, H, da))
        ion = np.append(ion, Ion_x(NHe, NH, H, He, He, H, spec=da, N=True))
    
    f = open('/Users/chang/X-ray-chemistry/KROME/AGN_pc/rate_dis/ratexHe' + str(int(da.distance_kpc))+'.dat', 'w')
    f.write('# KROME file, XRAY He rate\n')
    f.write('# Distance = ' + str(int(da.distance_kpc)) + ' kpc\n')
    f.write('# Xray rate including primary He ionization in 1/s\n')
    f.write('# first line indicates the size of the table (30x30)\n')
    f.write('# then first column is the parameter log10(nH), where nH is the H column density in cm-2\n')
    f.write('# second column is log10(xHe), where xHe = nHe/nH both in cm-2\n')
    f.write('# third column is log10(rate), where rate is in units of 1/s\n')
    f.write('30, 30')
    for i in range(len(ion)):
        if i%30==0:
            f.write('\n')
        f.writelines([str(dataH[i, 0]) + ' ' + str(dataH[i,1]) + ' ' + str(np.log10(ion[i])) + '\n'])
    f.close()

************************
alpha =  0.1
beta =  100
mdot =  3.0
************************
8.0
************************
alpha =  0.1
beta =  100
mdot =  3.0
************************
7.0
************************
alpha =  0.1
beta =  100
mdot =  3.0
************************
6.0
************************
alpha =  0.1
beta =  100
mdot =  3.0
************************
5.0
************************
alpha =  0.1
beta =  100
mdot =  3.0
************************
4.0
************************
alpha =  0.1
beta =  100
mdot =  3.0
************************
3.0
************************
alpha =  0.1
beta =  100
mdot =  3.0
************************
2.0
************************
alpha =  0.1
beta =  100
mdot =  3.0
************************
1.0


In [10]:
np.log10(num2col(2e4,2e4*0.09))

array([22.29847969, 21.2527222 ])

In [11]:
num2col(2e4,1.8e3)

(1.9882898458212567e+22, 1.789460861239131e+21)